In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
filename = "sap_storing_data_hu_project.csv" 
n = sum(1 for line in open(filename, encoding='utf8'))-1  
s = n//10
skip = sorted(random.sample(range(1, n+1), n-s))
df_not_cleaned = pd.read_csv(filename, skiprows=skip, low_memory=False)
#verwijder alle kolommen die niet van gebruik zijn:
df = df_not_cleaned.drop(columns=['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld', 'stm_geo_mld_uit_functiepl', 'stm_scenario_mon', 'stm_status_melding_sap', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_aanntpl_ddt', 'stm_geo_gst_uit_functiepl', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt', 'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz', 'stm_veroorz_groep', 'stm_veroorz_code', 'stm_progfh_gw_duur', 'stm_progfh_gw_lwd_tijd', 'stm_progfh_gw_lwd_datum', 'stm_progfh_gw_datum', 'stm_arbeid', 'stm_functiepl_gst', 'stm_functiepl_mld', 'stm_mon_begintijd', 'stm_mon_begindatum', 'stm_aangelegd_tijd', 'stm_aangelegd_dd', 'stm_dir_betrok_tr', 'stm_evb', 'stm_mon_vhdsincident', 'stm_mon_eind_ddt', 'stm_afspr_aanvangddt', 'stm_effect', 'stm_veroorz_tekst_kort', 'stm_projnr', 'stm_wbi_nummer', 'stm_status_gebr', 'stm_standplaats', 'stm_actie', 'stm_hinderwaarde', 'stm_rec_toegev_ddt', 'stm_afsluit_tijd', 'stm_afsluit_dd', 'stm_afsluit_ddt', 'stm_akkoord_melding_jn', 'stm_fact_jn', 'stm_status_sapnaarmon', 'stm_akkoord_mon_toewijz', 'stm_controle_dd', 'stm_mon_eind_tijd', 'stm_mon_eind_datum', 'stm_reactie_duur', 'stm_afspr_aanvangtijd', 'stm_afspr_aanvangdd', 'stm_historie_toelichting', 'stm_schade_verhaalb_jn', 'stm_x_bron_arch_dt', 'stm_x_bron_bestandsnaam', 'stm_x_bron_publ_dt', 'stm_rapportage_jaar', 'stm_rapportage_maand', 'stm_sorteerveld', 'stm_afspr_func_hersteltijd', 'stm_afspr_func_hersteldd', 'stm_relatiervo', 'stm_relatiervo_volgend', 'stm_relatiervo_vorig', 'stm_schade_statusdatum', 'stm_schade_status_ga', 'stm_schadenr', 'stm_x_start_sessie_dt', 'stm_x_bk', 'stm_x_run_id', 'stm_x_actueel_ind', 'stm_x_vervallen_ind'])
print("Done")

Done


# Data cleaning:


De te bepalen welke variabelen het meest van toepassing zijn om te gebruiken kijken we naar welke
kolommen informatie geven over de tijd van de melding, monteur. Tijd wanneer de melding word opgelost.
Wat voor melding het is en info over de melding/storing, en de locatie. Zo zijn de volgende kolommen
het meest van toepassing:

Over soort melding
- stm_sap_meldtekst (melding omschrijving)
- stm_geo_mld (melding code) - nodig
- stm_oorz_code (oorzaak code) - nodig
- stm_prioriteit (prioriteit)

Over tijd:
- stm_sap_melddatum (datump melding) - nodig
- stm_sap_meldtijd (tijdstip melding) - nodig
- stm_aanngeb_dd (datum aannemer gebeld)
- stm_aanngeb_tijd (tijd aannemer gebeld)
- stm_aanntpl_dd (datum aannemer op plek)
- stm_aanntpl_tijd (tijd aannemer op plek)

- stm_fh_dd (Datum van functie herstel) - nodig
- stm_fh_tijd (Tijdstip van functieherstel) - nodig
- stm_sap_storeinddatum (Datum einde storing) - nodig
- stm_sap_storeindtijd (Tijdstip einde storing) - nodig
- stm_fh_duur (Duur van functiehersteltijd) - nodig

Over equipment:
- stm_equipm_nr_mld (equipment)
- stm_equipm_omschr_mld (equipment omschrijving)

Over locatie
- stm_km_van_mld (locatie van)
- stm_km_tot_mld (locatie tot)
- stm_vl_post (VL post) - nodig
- stm_contractgeb_mld(locatie melding) - nodig
- stm_pplg_van (PPLG van locatie)
- stm_pplg_naar (PPLG tot locatie)
- stm_dstrglp_van (Dienstregelpunt van locatie)
- stm_dstrglp_naar (Dienstregelpunt tot locatie)

Over prognose:
- stm_progfh_in_datum (Datum prognose)
- stm_progfh_in_tijd (Tijdstip FHT prognose)
- stm_progfh_in_invoer_dat (Invoer datum prognose)
- stm_progfh_in_invoer_tijd (Invoer tijdstip FHT prognose)
- stm_progfh_in_duur (Duur FHT)



In [3]:
#deze functie verwijdert alle rijen waar geen value op de gegeven kolommen zitten
def drop_rows_with_null(lst, df):
    dfisnull = df.isnull()[lst]
    matches = dfisnull[(dfisnull).any(axis=1)]
    return df.drop(matches.index)

In [155]:
#Deze functie berkent de uitschiet grenzen
def uitschieters_bereken(kolom):
    q1 = kolom.quantile(0.25)
    q3 = kolom.quantile(0.75)
    iqr = q3 - q1
    maxuitschieter = q3 + (iqr*1.5)
    minuitschieter = q1 - (iqr*1.5)
    return maxuitschieter, minuitschieter

#Deze functie geeft de indexes van uitschieters in een kolom
def return_uitschieters_indexes(kolom):
    maxuitschieter = uitschieters_bereken(kolom)[0]
    minuitschieter = uitschieters_bereken(kolom)[1]
    uitschieters = [kolom[kolom > maxuitschieter].index, kolom[kolom < minuitschieter].index]
    return uitschieters
    
#Deze functie geeft een kolom zonder uitschieters
def return_kolom_zonder_uitschieters(kolom):
    maxuitschieter = uitschieters_bereken(kolom)[0]
    minuitschieter = uitschieters_bereken(kolom)[1]
    kolom = kolom[kolom < maxuitschieter][kolom > minuitschieter]
    return kolom


In [156]:
#Deze functie berkent de extreme uitschiet grenzen
def ex_uitschieters_bereken(kolom):
    q1 = kolom.quantile(0.25)
    q3 = kolom.quantile(0.75)
    iqr = q3 - q1
    maxuitschieter = q3 + (iqr*3)
    minuitschieter = q1 - (iqr*3)
    return maxuitschieter, minuitschieter

#Deze functie geeft de indexes van extreme uitschieters in een kolom
def return_ex_uitschieters_indexes(kolom):
    maxuitschieter = uitschieters_ex_bereken(kolom)[0]
    minuitschieter = uitschieters_ex_bereken(kolom)[1]
    uitschieters = [kolom[kolom > maxuitschieter].index, kolom[kolom < minuitschieter].index]
    return uitschieters
    
#Deze functie geeft een kolom zonder extreme uitschieters
def return_kolom_zonder_ex_uitschieters(kolom):
    maxuitschieter = uitschieters_ex_bereken(kolom)[0]
    minuitschieter = uitschieters_ex_bereken(kolom)[1]
    kolom = kolom[kolom < maxuitschieter][kolom > minuitschieter]
    return kolom



In [4]:
# Verwijder de rijen die geen value bevatten in de benodigde kolommen
df_filtered = drop_rows_with_null(['stm_geo_mld', 'stm_sap_melddatum', 'stm_oorz_code', 'stm_sap_meldtijd',
                          'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd',
                          'stm_fh_duur', 'stm_vl_post', 'stm_contractgeb_mld'], df)
print(len(df))
print(len(df_filtered))
df_filtered

89852
13285


,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_sap_meldtekst,stm_geo_mld,stm_equipm_nr_mld,stm_equipm_soort_mld,...,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_oorz_tekst_kort,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar
18,208,50053591,39.0,EHV,20/01/2006 14:32:00,Logboeknr RBV : 39 Tijd: 1432 VL-Post: ...,Logboeknr RBV : 39 Tijd: 1432 VL-P,515.0,10527838.0,WISSEL,...,20/01/2006,14:47:00,15.0,23/01/2006,11:39:00,Geen SMC-melding.,NaN,NaN,Vga,NaN
22,243,50053652,42.0,RSD,25/01/2006 09:30:00,Logboeknr RBV : 42 Tijd: 0930 VL-Post: ...,Logboeknr RBV : 42 Tijd: 0930 VL-P,127.0,10621371.0,DETECTREIN,...,25/01/2006,09:44:00,14.0,30/01/2006,11:21:00,Geen SMC-melding.Geen onderzoek geweest.,NaN,NaN,Arn,NaN
31,327,50053757,151.0,EHV,31/01/2006 14:51:00,Logboeknr RBV : 151 Tijd: 1451 VL-Post: ...,Logboeknr RBV : 151 Tijd: 1451 VL-P,53.0,10621339.0,DETECTREIN,...,31/01/2006,15:06:00,15.0,01/02/2006,09:32:00,Geen SMC-melding.,NaN,NaN,Btl,NaN
33,366,50053809,13.0,GVC,05/02/2006 06:32:00,Logboeknr RBV : 13 Tijd: 0632 VL-Post: ...,Logboeknr RBV : 13 Tijd: 0632 VL-P,112.0,10107603.0,BAANLICHAA,...,05/02/2006,07:18:00,46.0,05/02/2006,07:18:00,NaN,NaN,NaN,Dt,NaN
43,506,50054024,51.0,GVC,20/02/2006 09:51:00,Logboeknr RBV : 51 Tijd: 0951 VL-Post: ...,aanrijding (wegverkeer) op 9/2/6 met kn,103.0,10021034.0,OVERWEG,...,20/02/2006,10:39:00,48.0,20/02/2006,10:39:00,NaN,NaN,NaN,Ld,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89820,908236,99003005,974425.0,RTD,21/10/2010 19:42:00,"Dtz-Dt : Km 70.3-69.8, rijplaten waaien omhoo...","Dtz-Dt : Km 70.3-69.8, rijplaten waaien",112.0,10107602.0,BAANLICHAA,...,21/10/2010,20:59:00,77.0,21/10/2010,21:00:00,NaN,SDM,NaN,SDM,DTZ
89832,908413,99003167,1994752.0,RTD,09/05/2013 20:33:00,Brd : [2+] Brandmelding Station 2 sprinklerpo...,Brd : [2+] Brandmelding Station 2 sprink,116.0,10590754.0,TUNCONSTR,...,09/05/2013,21:11:00,37.0,09/05/2013,21:30:00,NaN,KFHAZ,NaN,KFHAZ,KFHAZ
89834,908436,99003178,2274362.0,UT,19/11/2013 22:58:00,Md-Nwk : Ovw 36.8 boom blijft dicht liggen.,Md-Nwk : Ovw 36.8 boom blijft dicht ligg,132.0,10247772.0,OVERWEG,...,20/12/2013,00:15:00,43217.0,20/12/2013,00:33:00,Status gewijzigd van Concept naar Compl,NaN,NWKI,MDO,00/00/0000
89840,908496,99003200,2434924.0,RTD,15/03/2014 14:04:00,"Ldm : T.o.b.s. kortstondig (1x), S-76 afgeval...","Ldm : T.o.b.s. kortstondig (1x), S-76 af",118.0,10621361.0,DETECTREIN,...,17/03/2014,09:45:00,1.0,17/03/2014,09:45:00,Status gewijzigd van Concept naar Compl,NaN,LDM,LDM,00/00/0000


In [5]:
#uitgeschreven lijsten zodat ik ze niet steeds hoef over te typen

# melding
df[['stm_sap_meldtekst', 'stm_geo_mld', 'stm_oorz_code', 'stm_prioriteit']]
# tijd
df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd']]
df[['stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']]
# equipment
df[['stm_equipm_omschr_mld', 'stm_equipm_nr_mld']]
# locatie
df[['stm_km_van_mld', 'stm_km_tot_mld', 'stm_vl_post', 'stm_contractgeb_mld', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar']]
# prognose
df[['stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur']]


,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur
0,NaN,00:00:00,NaN,00:00:00,99999999.0
1,NaN,00:00:00,NaN,00:00:00,99999999.0
2,NaN,00:00:00,NaN,00:00:00,99999999.0
3,NaN,00:00:00,NaN,00:00:00,99999999.0
4,NaN,00:00:00,NaN,00:00:00,99999999.0
...,...,...,...,...,...
89847,10/05/2013,22:50:00,10/05/2013,22:59:22,71
89848,NaN,00:00:00,NaN,00:00:00,99999999
89849,NaN,00:00:00,NaN,00:00:00,99999999
89850,11/05/2013,05:30:00,11/05/2013,05:26:35,159


# Baseline
Alle problemen worden in 3 uur opgelost.

Van de 661090 meldingen worden er 295471 opgelost binnen 3 uur dat is 44.7%. 
van de 898526 melding worden er 618496 opgelost binnen 3 uur tijd dat is 68.8%.

In [18]:
onder_3_uur = df['stm_fh_duur'].dropna()
len(onder_3_uur.loc[(onder_3_uur < 180)])
#len(df)


61936

In [19]:
baseline_2 = 100/898526*618496
print('baseline model 2 :', baseline_2)

baseline model 2 : 68.83451341419169


In [6]:
begin_eind_tijd_dirty = df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']].dropna()
# begin en eindtijd in dataframe gezet.

In [7]:
begin_eind_tijd_clean = begin_eind_tijd_dirty.loc[(begin_eind_tijd_dirty['stm_sap_meldtijd'] != '::') & (begin_eind_tijd_dirty['stm_sap_storeindtijd'] != '::')]

In [8]:
stm_sap_melddatum_tijd_object = begin_eind_tijd_clean['stm_sap_melddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_meldtijd']
stm_sap_melddatum_tijd = pd.DataFrame(stm_sap_melddatum_tijd_object,
                   columns=['Begin_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [9]:
stm_sap_einddatum_tijd_object = begin_eind_tijd_clean['stm_sap_storeinddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_storeindtijd']
stm_sap_einddatum_tijd = pd.DataFrame(stm_sap_einddatum_tijd_object,
                   columns=['Eind_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [10]:
result_begin_eind_tijd = pd.concat([begin_eind_tijd_clean, stm_sap_melddatum_tijd, stm_sap_einddatum_tijd],axis=1).dropna()
result_begin_eind_tijd # de samengevoegde waarde in de dataframe gezet

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
0,02/01/2006,23:25:00,03/01/2006,09:44:00,02/01/2006 23:25:00,03/01/2006 09:44:00
1,03/01/2006,19:18:00,03/01/2006,19:42:00,03/01/2006 19:18:00,03/01/2006 19:42:00
2,03/01/2006,15:19:00,04/01/2006,09:38:00,03/01/2006 15:19:00,04/01/2006 09:38:00
3,04/01/2006,21:01:00,04/01/2006,21:21:00,04/01/2006 21:01:00,04/01/2006 21:21:00
4,05/01/2006,10:50:00,09/01/2006,09:10:00,05/01/2006 10:50:00,09/01/2006 09:10:00
...,...,...,...,...,...,...
89845,10/05/2013,17:54:00,10/05/2013,20:40:00,10/05/2013 17:54:00,10/05/2013 20:40:00
89846,10/05/2013,18:39:00,03/07/2013,05:42:00,10/05/2013 18:39:00,03/07/2013 05:42:00
89847,10/05/2013,19:35:00,10/05/2013,22:50:00,10/05/2013 19:35:00,10/05/2013 22:50:00
89849,11/05/2013,00:27:00,11/05/2013,01:07:00,11/05/2013 00:27:00,11/05/2013 01:07:00


In [11]:
result_begin_eind_tijd['Begin_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Begin_tijd_en_datum'])
result_begin_eind_tijd['Eind_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Eind_tijd_en_datum'])

In [12]:
result_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
0,02/01/2006,23:25:00,03/01/2006,09:44:00,2006-02-01 23:25:00,2006-03-01 09:44:00
1,03/01/2006,19:18:00,03/01/2006,19:42:00,2006-03-01 19:18:00,2006-03-01 19:42:00
2,03/01/2006,15:19:00,04/01/2006,09:38:00,2006-03-01 15:19:00,2006-04-01 09:38:00
3,04/01/2006,21:01:00,04/01/2006,21:21:00,2006-04-01 21:01:00,2006-04-01 21:21:00
4,05/01/2006,10:50:00,09/01/2006,09:10:00,2006-05-01 10:50:00,2006-09-01 09:10:00
...,...,...,...,...,...,...
89845,10/05/2013,17:54:00,10/05/2013,20:40:00,2013-10-05 17:54:00,2013-10-05 20:40:00
89846,10/05/2013,18:39:00,03/07/2013,05:42:00,2013-10-05 18:39:00,2013-03-07 05:42:00
89847,10/05/2013,19:35:00,10/05/2013,22:50:00,2013-10-05 19:35:00,2013-10-05 22:50:00
89849,11/05/2013,00:27:00,11/05/2013,01:07:00,2013-11-05 00:27:00,2013-11-05 01:07:00


In [13]:
fix_duur = result_begin_eind_tijd['Eind_tijd_en_datum'] - result_begin_eind_tijd['Begin_tijd_en_datum']

In [14]:
Duur_fix = pd.DataFrame(fix_duur,
                   columns=['Duur fix'])

In [15]:
Final_begin_eind_tijd = pd.concat([result_begin_eind_tijd, Duur_fix],axis=1).dropna()
Final_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
0,02/01/2006,23:25:00,03/01/2006,09:44:00,2006-02-01 23:25:00,2006-03-01 09:44:00,27 days 10:19:00
1,03/01/2006,19:18:00,03/01/2006,19:42:00,2006-03-01 19:18:00,2006-03-01 19:42:00,0 days 00:24:00
2,03/01/2006,15:19:00,04/01/2006,09:38:00,2006-03-01 15:19:00,2006-04-01 09:38:00,30 days 18:19:00
3,04/01/2006,21:01:00,04/01/2006,21:21:00,2006-04-01 21:01:00,2006-04-01 21:21:00,0 days 00:20:00
4,05/01/2006,10:50:00,09/01/2006,09:10:00,2006-05-01 10:50:00,2006-09-01 09:10:00,122 days 22:20:00
...,...,...,...,...,...,...,...
89845,10/05/2013,17:54:00,10/05/2013,20:40:00,2013-10-05 17:54:00,2013-10-05 20:40:00,0 days 02:46:00
89846,10/05/2013,18:39:00,03/07/2013,05:42:00,2013-10-05 18:39:00,2013-03-07 05:42:00,-213 days +11:03:00
89847,10/05/2013,19:35:00,10/05/2013,22:50:00,2013-10-05 19:35:00,2013-10-05 22:50:00,0 days 03:15:00
89849,11/05/2013,00:27:00,11/05/2013,01:07:00,2013-11-05 00:27:00,2013-11-05 01:07:00,0 days 00:40:00


In [16]:
Final_begin_eind_tijd.loc[(Final_begin_eind_tijd['Duur fix'] > '0 days 00:00:00') & (Final_begin_eind_tijd['Duur fix'] < '0 days 03:00:00')]

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
1,03/01/2006,19:18:00,03/01/2006,19:42:00,2006-03-01 19:18:00,2006-03-01 19:42:00,00:24:00
3,04/01/2006,21:01:00,04/01/2006,21:21:00,2006-04-01 21:01:00,2006-04-01 21:21:00,00:20:00
5,07/01/2006,00:30:00,07/01/2006,00:40:00,2006-07-01 00:30:00,2006-07-01 00:40:00,00:10:00
6,09/01/2006,14:55:00,09/01/2006,15:13:00,2006-09-01 14:55:00,2006-09-01 15:13:00,00:18:00
7,10/01/2006,16:54:00,10/01/2006,17:36:00,2006-10-01 16:54:00,2006-10-01 17:36:00,00:42:00
...,...,...,...,...,...,...,...
89842,23/04/2016,23:50:07,24/04/2016,00:14:00,2016-04-23 23:50:07,2016-04-24 00:14:00,00:23:53
89843,20/02/2016,13:29:38,20/02/2016,14:38:34,2016-02-20 13:29:38,2016-02-20 14:38:34,01:08:56
89845,10/05/2013,17:54:00,10/05/2013,20:40:00,2013-10-05 17:54:00,2013-10-05 20:40:00,02:46:00
89849,11/05/2013,00:27:00,11/05/2013,01:07:00,2013-11-05 00:27:00,2013-11-05 01:07:00,00:40:00


In [17]:
#
baseline_score= 100/661090*295471
print('dit is de baseline score:',baseline_score)

dit is de baseline score: 44.694519656930225
